In [0]:
from math import isnan
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import pandas as pd

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# Load the data

In [3]:
root_path = 'gdrive/My Drive/Capstone_ML/'
merged_features = pd.read_csv(root_path+'MariasWorkingFolder/merged_features.csv')
merged_features.head()

,Unnamed: 0,YEAR,MONTH,LATITUDE,LONGITUDE,NOM_ARROND,NON_FIRE_EVENT_COUNT,FIRE_EVENT_COUNT,FIRE_EVENT,Résidentiel_BUILDING_COUNT,Résidentiel_STORY_COUNT,Résidentiel_UNIT_COUNT,Résidentiel_AGE_AVG,Résidentiel_LAND_AREA_TOTAL,Résidentiel_BUILDING_AREA_TOTAL,Commercial_BUILDING_COUNT,Commercial_STORY_COUNT,Commercial_UNIT_COUNT,Commercial_AGE_AVG,Commercial_LAND_AREA_TOTAL,Commercial_BUILDING_AREA_TOTAL,Terrain vague_BUILDING_COUNT,Terrain vague_LAND_AREA_TOTAL,Terrain vague_BUILDING_AREA_TOTAL,Ruelle_BUILDING_COUNT,Ruelle_LAND_AREA_TOTAL,Industriel_BUILDING_COUNT,Industriel_STORY_COUNT,Industriel_UNIT_COUNT,Industriel_AGE_AVG,Industriel_LAND_AREA_TOTAL,Industriel_BUILDING_AREA_TOTAL,Transport_BUILDING_COUNT,Transport_STORY_COUNT,Transport_UNIT_COUNT,Transport_AGE_AVG,Transport_LAND_AREA_TOTAL,Transport_BUILDING_AREA_TOTAL,Parc_BUILDING_COUNT,Parc_STORY_COUNT,...,Immeuble inutilisé_BUILDING_COUNT,Immeuble inutilisé_STORY_COUNT,Immeuble inutilisé_AGE_AVG,Immeuble inutilisé_LAND_AREA_TOTAL,Immeuble inutilisé_BUILDING_AREA_TOTAL,Éducation_BUILDING_COUNT,Éducation_STORY_COUNT,Éducation_UNIT_COUNT,Éducation_AGE_AVG,Éducation_LAND_AREA_TOTAL,Éducation_BUILDING_AREA_TOTAL,Loisirs_BUILDING_COUNT,Loisirs_STORY_COUNT,Loisirs_UNIT_COUNT,Loisirs_AGE_AVG,Loisirs_LAND_AREA_TOTAL,Loisirs_BUILDING_AREA_TOTAL,Santé_BUILDING_COUNT,Santé_STORY_COUNT,Santé_UNIT_COUNT,Santé_AGE_AVG,Santé_LAND_AREA_TOTAL,Santé_BUILDING_AREA_TOTAL,Sécurité publique_BUILDING_COUNT,Sécurité publique_STORY_COUNT,Sécurité publique_AGE_AVG,Sécurité publique_LAND_AREA_TOTAL,Sécurité publique_BUILDING_AREA_TOTAL,Restaurant_BUILDING_COUNT,Restaurant_STORY_COUNT,Restaurant_UNIT_COUNT,Restaurant_AGE_AVG,Restaurant_LAND_AREA_TOTAL,Restaurant_BUILDING_AREA_TOTAL,Étudiants_BUILDING_COUNT,Étudiants_STORY_COUNT,Étudiants_UNIT_COUNT,Étudiants_AGE_AVG,Étudiants_LAND_AREA_TOTAL,Étudiants_BUILDING_AREA_TOTAL
0,0,2005,1,45.62,-73.62,Montréal-Nord,33,8,1,12424,22912.0,38621.0,41.689070,4774185.0,3375618.0,373,320.0,10.0,36.528150,1157943.0,429020.0,266,501938.0,8471.0,19,886.0,73,44.0,0.0,30.808219,280703.0,101218.0,258,134.0,3.0,36.116279,368366.0,96594.0,19,1.0,...,5,1.0,27.800000,3250.0,621.0,46,57.0,9.0,43.130435,355991.0,210481.0,3,0.0,0.0,25.666667,63256.0,6022.0,2,1.0,0.0,34.00,12369.0,12007.0,2,0.0,15.000000,28972.0,2549.0,6,7.0,0.0,18.000000,12172.0,1517.0,1,4.0,5.0,9.000000,5220.0,0.0
1,1,2005,1,45.49,-73.58,Ville-Marie / Parc Jean-Drapeau / Centre-Sud,35,24,1,21903,26711.0,54682.0,51.711409,2537161.0,16476280.0,2373,3348.0,297.0,55.383059,1706791.0,7617611.0,2009,860141.0,89618.0,2,1198.0,41,55.0,6.0,53.853659,846167.0,167834.0,198,50.0,14.0,19.530303,2464043.0,56181.0,65,6.0,...,10,33.0,80.300000,59627.0,14833.0,88,170.0,42.0,55.625000,511539.0,910873.0,40,26.0,4.0,56.625000,539817.0,137906.0,24,13.0,0.0,59.25,276251.0,285261.0,6,8.0,61.833333,24472.0,15553.0,46,43.0,1.0,65.695652,10946.0,89167.0,13,70.0,26.0,62.076923,48131.0,125131.0
2,2,2005,1,45.48,-73.69,St-Laurent,2,2,1,18856,23343.0,35527.0,29.361423,5763462.0,9377079.0,726,723.0,5.0,29.013774,8090310.0,2719256.0,715,3894999.0,39219.0,18,5182.0,240,121.0,0.0,22.375000,3692195.0,1254622.0,579,423.0,0.0,26.350604,9890935.0,1093602.0,48,2.0,...,0,0.0,0.000000,0.0,0.0,59,51.0,2.0,43.559322,741242.0,186990.0,6,8.0,2.0,49.333333,80889.0,35399.0,2,3.0,0.0,37.00,4667.0,844.0,3,0.0,6.333333,29704.0,1789.0,8,8.0,0.0,23.125000,21994.0,1516.0,3,7.0,165.0,39.000000,4991.0,102.0
3,3,2005,1,45.54,-73.55,Mercier / Hochelaga-Maisonneuve,8,3,1,25257,44836.0,66187.0,49.637012,6780004.0,9244519.0,368,392.0,26.0,45.290761,1881853.0,1633321.0,755,1540265.0,25553.0,9,10843.0,57,67.0,0.0,45.894737,3197240.0,566205.0,95,41.0,10.0,27.863158,964932.0,66015.0,39,7.0,...,3,2.0,40.666667,16564.0,848.0,80,58.0,2.0,40.162500,753976.0,261331.0,15,12.0,18.0,44.000000,171880.0,28484.0,10,5.0,6.0,42.90,330657.0,131033.0,6,4.0,57.166667,860673.0,221951.0,26,25.0,0.0,35.923077,41118.0,8020.0,0,0.0,0.0,0.000000,0.0,0.0
4,4,2005,1,45.61,-73.63,Montréal-Nord,9,

# Final cleaning

## Factorizing string data

In [4]:
merged_features['NOM_ARROND'], unique_arronds = pd.factorize(merged_features['NOM_ARROND'])
unique_arronds

Index(['Montréal-Nord', 'Ville-Marie / Parc Jean-Drapeau / Centre-Sud',
       'St-Laurent', 'Mercier / Hochelaga-Maisonneuve', 'Anjou', 'Westmount',
       'Plateau Mont-Royal', 'Rivière-des-Prairies / Pointe-aux-Trembles',
       'Ahuntsic / Cartierville', 'St-Léonard', 'Rosemont / Petite-Patrie',
       'Ile-Bizard / Ste-Geneviève / Ste-A-de-B',
       'Villeray / St-Michel / Parc Extension', 'Verdun', 'Sud-Ouest',
       'Lasalle', 'Côte-des-Neiges / Notre-Dame-de-Grâce', 'Mont-Royal',
       'Dollard-des-Ormeaux / Roxboro', 'Pierrefonds / Roxboro / Senneville',
       'Pointe-Claire', 'Côte St-Luc / Hampstead / Mtl-Ouest',
       'Dorval / Ile Dorval', 'Beaconsfield / Baie d'Urfé', 'Lachine',
       'Outremont', 'Kirkland', 'Montréal-Est'],
      dtype='object')

In [5]:
merged_features.head()

,Unnamed: 0,YEAR,MONTH,LATITUDE,LONGITUDE,NOM_ARROND,NON_FIRE_EVENT_COUNT,FIRE_EVENT_COUNT,FIRE_EVENT,Résidentiel_BUILDING_COUNT,Résidentiel_STORY_COUNT,Résidentiel_UNIT_COUNT,Résidentiel_AGE_AVG,Résidentiel_LAND_AREA_TOTAL,Résidentiel_BUILDING_AREA_TOTAL,Commercial_BUILDING_COUNT,Commercial_STORY_COUNT,Commercial_UNIT_COUNT,Commercial_AGE_AVG,Commercial_LAND_AREA_TOTAL,Commercial_BUILDING_AREA_TOTAL,Terrain vague_BUILDING_COUNT,Terrain vague_LAND_AREA_TOTAL,Terrain vague_BUILDING_AREA_TOTAL,Ruelle_BUILDING_COUNT,Ruelle_LAND_AREA_TOTAL,Industriel_BUILDING_COUNT,Industriel_STORY_COUNT,Industriel_UNIT_COUNT,Industriel_AGE_AVG,Industriel_LAND_AREA_TOTAL,Industriel_BUILDING_AREA_TOTAL,Transport_BUILDING_COUNT,Transport_STORY_COUNT,Transport_UNIT_COUNT,Transport_AGE_AVG,Transport_LAND_AREA_TOTAL,Transport_BUILDING_AREA_TOTAL,Parc_BUILDING_COUNT,Parc_STORY_COUNT,...,Immeuble inutilisé_BUILDING_COUNT,Immeuble inutilisé_STORY_COUNT,Immeuble inutilisé_AGE_AVG,Immeuble inutilisé_LAND_AREA_TOTAL,Immeuble inutilisé_BUILDING_AREA_TOTAL,Éducation_BUILDING_COUNT,Éducation_STORY_COUNT,Éducation_UNIT_COUNT,Éducation_AGE_AVG,Éducation_LAND_AREA_TOTAL,Éducation_BUILDING_AREA_TOTAL,Loisirs_BUILDING_COUNT,Loisirs_STORY_COUNT,Loisirs_UNIT_COUNT,Loisirs_AGE_AVG,Loisirs_LAND_AREA_TOTAL,Loisirs_BUILDING_AREA_TOTAL,Santé_BUILDING_COUNT,Santé_STORY_COUNT,Santé_UNIT_COUNT,Santé_AGE_AVG,Santé_LAND_AREA_TOTAL,Santé_BUILDING_AREA_TOTAL,Sécurité publique_BUILDING_COUNT,Sécurité publique_STORY_COUNT,Sécurité publique_AGE_AVG,Sécurité publique_LAND_AREA_TOTAL,Sécurité publique_BUILDING_AREA_TOTAL,Restaurant_BUILDING_COUNT,Restaurant_STORY_COUNT,Restaurant_UNIT_COUNT,Restaurant_AGE_AVG,Restaurant_LAND_AREA_TOTAL,Restaurant_BUILDING_AREA_TOTAL,Étudiants_BUILDING_COUNT,Étudiants_STORY_COUNT,Étudiants_UNIT_COUNT,Étudiants_AGE_AVG,Étudiants_LAND_AREA_TOTAL,Étudiants_BUILDING_AREA_TOTAL
0,0,2005,1,45.62,-73.62,0,33,8,1,12424,22912.0,38621.0,41.689070,4774185.0,3375618.0,373,320.0,10.0,36.528150,1157943.0,429020.0,266,501938.0,8471.0,19,886.0,73,44.0,0.0,30.808219,280703.0,101218.0,258,134.0,3.0,36.116279,368366.0,96594.0,19,1.0,...,5,1.0,27.800000,3250.0,621.0,46,57.0,9.0,43.130435,355991.0,210481.0,3,0.0,0.0,25.666667,63256.0,6022.0,2,1.0,0.0,34.00,12369.0,12007.0,2,0.0,15.000000,28972.0,2549.0,6,7.0,0.0,18.000000,12172.0,1517.0,1,4.0,5.0,9.000000,5220.0,0.0
1,1,2005,1,45.49,-73.58,1,35,24,1,21903,26711.0,54682.0,51.711409,2537161.0,16476280.0,2373,3348.0,297.0,55.383059,1706791.0,7617611.0,2009,860141.0,89618.0,2,1198.0,41,55.0,6.0,53.853659,846167.0,167834.0,198,50.0,14.0,19.530303,2464043.0,56181.0,65,6.0,...,10,33.0,80.300000,59627.0,14833.0,88,170.0,42.0,55.625000,511539.0,910873.0,40,26.0,4.0,56.625000,539817.0,137906.0,24,13.0,0.0,59.25,276251.0,285261.0,6,8.0,61.833333,24472.0,15553.0,46,43.0,1.0,65.695652,10946.0,89167.0,13,70.0,26.0,62.076923,48131.0,125131.0
2,2,2005,1,45.48,-73.69,2,2,2,1,18856,23343.0,35527.0,29.361423,5763462.0,9377079.0,726,723.0,5.0,29.013774,8090310.0,2719256.0,715,3894999.0,39219.0,18,5182.0,240,121.0,0.0,22.375000,3692195.0,1254622.0,579,423.0,0.0,26.350604,9890935.0,1093602.0,48,2.0,...,0,0.0,0.000000,0.0,0.0,59,51.0,2.0,43.559322,741242.0,186990.0,6,8.0,2.0,49.333333,80889.0,35399.0,2,3.0,0.0,37.00,4667.0,844.0,3,0.0,6.333333,29704.0,1789.0,8,8.0,0.0,23.125000,21994.0,1516.0,3,7.0,165.0,39.000000,4991.0,102.0
3,3,2005,1,45.54,-73.55,3,8,3,1,25257,44836.0,66187.0,49.637012,6780004.0,9244519.0,368,392.0,26.0,45.290761,1881853.0,1633321.0,755,1540265.0,25553.0,9,10843.0,57,67.0,0.0,45.894737,3197240.0,566205.0,95,41.0,10.0,27.863158,964932.0,66015.0,39,7.0,...,3,2.0,40.666667,16564.0,848.0,80,58.0,2.0,40.162500,753976.0,261331.0,15,12.0,18.0,44.000000,171880.0,28484.0,10,5.0,6.0,42.90,330657.0,131033.0,6,4.0,57.166667,860673.0,221951.0,26,25.0,0.0,35.923077,41118.0,8020.0,0,0.0,0.0,0.000000,0.0,0.0
4,4,2005,1,45.61,-73.63,0,9,3,1,12424,22912.0,38621.0,41.689070,4774185.0,3375618.0,373,320.0,10.0,36.528150,1157943.0,429020.0,266,50

## Binning the risk of fires

In [6]:
merged_features['FIRE_EVENT_COUNT'].max()

59

In [7]:
# Each line is on a per month basis. So we define the bins as such:
#    1:Low (0 to 4 fire events per month)
#    2:Medium (5 to 19 fire events per month)
#    3:High (more than 20 fire events per month)
labels = [1,2,3]
bins = [-1, 4, 20, merged_features['FIRE_EVENT_COUNT'].max()]
merged_features['FIRE_RISK'] = pd.cut(merged_features['FIRE_EVENT_COUNT'], bins=bins, labels=labels)
merged_features.head()

,Unnamed: 0,YEAR,MONTH,LATITUDE,LONGITUDE,NOM_ARROND,NON_FIRE_EVENT_COUNT,FIRE_EVENT_COUNT,FIRE_EVENT,Résidentiel_BUILDING_COUNT,Résidentiel_STORY_COUNT,Résidentiel_UNIT_COUNT,Résidentiel_AGE_AVG,Résidentiel_LAND_AREA_TOTAL,Résidentiel_BUILDING_AREA_TOTAL,Commercial_BUILDING_COUNT,Commercial_STORY_COUNT,Commercial_UNIT_COUNT,Commercial_AGE_AVG,Commercial_LAND_AREA_TOTAL,Commercial_BUILDING_AREA_TOTAL,Terrain vague_BUILDING_COUNT,Terrain vague_LAND_AREA_TOTAL,Terrain vague_BUILDING_AREA_TOTAL,Ruelle_BUILDING_COUNT,Ruelle_LAND_AREA_TOTAL,Industriel_BUILDING_COUNT,Industriel_STORY_COUNT,Industriel_UNIT_COUNT,Industriel_AGE_AVG,Industriel_LAND_AREA_TOTAL,Industriel_BUILDING_AREA_TOTAL,Transport_BUILDING_COUNT,Transport_STORY_COUNT,Transport_UNIT_COUNT,Transport_AGE_AVG,Transport_LAND_AREA_TOTAL,Transport_BUILDING_AREA_TOTAL,Parc_BUILDING_COUNT,Parc_STORY_COUNT,...,Immeuble inutilisé_STORY_COUNT,Immeuble inutilisé_AGE_AVG,Immeuble inutilisé_LAND_AREA_TOTAL,Immeuble inutilisé_BUILDING_AREA_TOTAL,Éducation_BUILDING_COUNT,Éducation_STORY_COUNT,Éducation_UNIT_COUNT,Éducation_AGE_AVG,Éducation_LAND_AREA_TOTAL,Éducation_BUILDING_AREA_TOTAL,Loisirs_BUILDING_COUNT,Loisirs_STORY_COUNT,Loisirs_UNIT_COUNT,Loisirs_AGE_AVG,Loisirs_LAND_AREA_TOTAL,Loisirs_BUILDING_AREA_TOTAL,Santé_BUILDING_COUNT,Santé_STORY_COUNT,Santé_UNIT_COUNT,Santé_AGE_AVG,Santé_LAND_AREA_TOTAL,Santé_BUILDING_AREA_TOTAL,Sécurité publique_BUILDING_COUNT,Sécurité publique_STORY_COUNT,Sécurité publique_AGE_AVG,Sécurité publique_LAND_AREA_TOTAL,Sécurité publique_BUILDING_AREA_TOTAL,Restaurant_BUILDING_COUNT,Restaurant_STORY_COUNT,Restaurant_UNIT_COUNT,Restaurant_AGE_AVG,Restaurant_LAND_AREA_TOTAL,Restaurant_BUILDING_AREA_TOTAL,Étudiants_BUILDING_COUNT,Étudiants_STORY_COUNT,Étudiants_UNIT_COUNT,Étudiants_AGE_AVG,Étudiants_LAND_AREA_TOTAL,Étudiants_BUILDING_AREA_TOTAL,FIRE_RISK
0,0,2005,1,45.62,-73.62,0,33,8,1,12424,22912.0,38621.0,41.689070,4774185.0,3375618.0,373,320.0,10.0,36.528150,1157943.0,429020.0,266,501938.0,8471.0,19,886.0,73,44.0,0.0,30.808219,280703.0,101218.0,258,134.0,3.0,36.116279,368366.0,96594.0,19,1.0,...,1.0,27.800000,3250.0,621.0,46,57.0,9.0,43.130435,355991.0,210481.0,3,0.0,0.0,25.666667,63256.0,6022.0,2,1.0,0.0,34.00,12369.0,12007.0,2,0.0,15.000000,28972.0,2549.0,6,7.0,0.0,18.000000,12172.0,1517.0,1,4.0,5.0,9.000000,5220.0,0.0,2
1,1,2005,1,45.49,-73.58,1,35,24,1,21903,26711.0,54682.0,51.711409,2537161.0,16476280.0,2373,3348.0,297.0,55.383059,1706791.0,7617611.0,2009,860141.0,89618.0,2,1198.0,41,55.0,6.0,53.853659,846167.0,167834.0,198,50.0,14.0,19.530303,2464043.0,56181.0,65,6.0,...,33.0,80.300000,59627.0,14833.0,88,170.0,42.0,55.625000,511539.0,910873.0,40,26.0,4.0,56.625000,539817.0,137906.0,24,13.0,0.0,59.25,276251.0,285261.0,6,8.0,61.833333,24472.0,15553.0,46,43.0,1.0,65.695652,10946.0,89167.0,13,70.0,26.0,62.076923,48131.0,125131.0,3
2,2,2005,1,45.48,-73.69,2,2,2,1,18856,23343.0,35527.0,29.361423,5763462.0,9377079.0,726,723.0,5.0,29.013774,8090310.0,2719256.0,715,3894999.0,39219.0,18,5182.0,240,121.0,0.0,22.375000,3692195.0,1254622.0,579,423.0,0.0,26.350604,9890935.0,1093602.0,48,2.0,...,0.0,0.000000,0.0,0.0,59,51.0,2.0,43.559322,741242.0,186990.0,6,8.0,2.0,49.333333,80889.0,35399.0,2,3.0,0.0,37.00,4667.0,844.0,3,0.0,6.333333,29704.0,1789.0,8,8.0,0.0,23.125000,21994.0,1516.0,3,7.0,165.0,39.000000,4991.0,102.0,1
3,3,2005,1,45.54,-73.55,3,8,3,1,25257,44836.0,66187.0,49.637012,6780004.0,9244519.0,368,392.0,26.0,45.290761,1881853.0,1633321.0,755,1540265.0,25553.0,9,10843.0,57,67.0,0.0,45.894737,3197240.0,566205.0,95,41.0,10.0,27.863158,964932.0,66015.0,39,7.0,...,2.0,40.666667,16564.0,848.0,80,58.0,2.0,40.162500,753976.0,261331.0,15,12.0,18.0,44.000000,171880.0,28484.0,10,5.0,6.0,42.90,330657.0,131033.0,6,4.0,57.166667,860673.0,221951.0,26,25.0,0.0,35.923077,41118.0,8020.0,0,0.0,0.0,0.000000,0.0,0.0,1
4,4,2005,1,45.61,-73.63,0,9,3,1,12424,22912.0,38621.0,41.689070,4774185.0,3375618.0,373,320.0,10.0,36.528150,1157943.0,429020.0,266,501938.0,8471.0,19,886.0,73

In [8]:
# Making sure all the data was binned correctly
merged_features['FIRE_RISK'].isna().sum()

0

## Removing some undesired columns

In [0]:
# It would require another model model to predict the other features and use them to make a fire risk prediction, so we can't use them for training!
merged_features = merged_features.drop(['Unnamed: 0', 'NON_FIRE_EVENT_COUNT', 'FIRE_EVENT_COUNT', 'FIRE_EVENT'], axis=1)

# Prepare training and testing data

In [0]:
merged_features_before_2018 = merged_features[merged_features['YEAR'] < 2018.0]
merged_features_from_2018 = merged_features[merged_features['YEAR'] == 2018.0]

# We're dropping the year in the training and test set as we don't want our model to be biased on that information

train_X = merged_features_before_2018.drop(['YEAR', 'FIRE_RISK'], axis=1)
train_y = merged_features_before_2018[['FIRE_RISK']]

test_X = merged_features_from_2018.drop(['YEAR', 'FIRE_RISK'], axis=1)
test_y = merged_features_from_2018[['FIRE_RISK']]

In [11]:
# Making sure there is something in here
len(test_y)

6694

# Model : Random Forest Classifier

In [61]:
rf = RandomForestClassifier(200, max_depth=10, class_weight='balanced')
rf.fit(train_X, train_y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=200, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [62]:
rf_pred = rf.predict(test_X)
accuracy_score(test_y, rf_pred)

ValueError: ignored

In [41]:
features = []
for col in train_X.columns: 
    features.append(col)

feat_list = []
for feature in zip(features, rf.feature_importances_):
      feat_list.append(feature)

def sortSecond(val): 
    return val[1] 
feat_list.sort(key = sortSecond, reverse = True)  
#print(feat_list)

for feat in feat_list:
  if(feat[1] > 0.015):
    print(feat)

('LONGITUDE', 0.0979307772261398)
('LATITUDE', 0.08768749789130696)
('Loisirs_BUILDING_COUNT', 0.060631983732818105)
('Construction_STORY_COUNT', 0.05718964502113589)
('Étudiants_BUILDING_AREA_TOTAL', 0.05465679453184524)
('Santé_BUILDING_COUNT', 0.047336501925531016)
('Commercial_UNIT_COUNT', 0.04391032799642418)
('Étudiants_LAND_AREA_TOTAL', 0.04008259285136193)
('Étudiants_BUILDING_COUNT', 0.03373050981120487)
('Étudiants_STORY_COUNT', 0.03033069646295544)
('MONTH', 0.026565313899785975)
('Lieu de culte_AGE_AVG', 0.019353091238361587)
('Industriel_UNIT_COUNT', 0.019029217800681016)
('Commercial_BUILDING_COUNT', 0.018583265292720533)
('Éducation_STORY_COUNT', 0.0172582859365844)
('Résidentiel_BUILDING_AREA_TOTAL', 0.01659575374297739)
('Sécurité publique_AGE_AVG', 0.016081102037297134)
('Industriel_AGE_AVG', 0.015520171371140666)


In [42]:
from sklearn.feature_selection import SelectFromModel

#Identify most important features and train selector
sel = SelectFromModel(rf, threshold=0.015)
sel.fit(train_X, train_y)
# Creating a dataframe from the selected features
#feature_idx = sel.get_support()
#feature_name = train_X.columns[feature_idx]
#X_train_sel_df = pd.DataFrame(X_train_sel)
#X_train_sel_df.columns = feature_name

#Create new dataset with most important features
X_train_sel = sel.transform(train_X)
X_test_sel = sel.transform(test_X)

rf_sel = RandomForestClassifier(200, max_depth=18)
rf_sel.fit(X_train_sel,train_y)

#Results
rf_sel_pred = rf_sel.predict(X_test_sel)
accuracy_score(test_y, rf_sel_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/from_model.py:196: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':


0.8564386017328951

# Predicting the 2019 risk of fire

In [0]:
test_X_2019 = test_X  # Data from 2018
for column_name in test_X_2019.columns:
  if column_name.endswith('_AGE_AVG'):
    test_X_2019[column_name] = test_X_2019[column_name] + 1  # Buildings will be 1 year older

In [0]:
rf_pred_2019 = rf.predict(test_X)

In [0]:
results_2019 = test_X_2019
results_2019['FIRE_RISK'] = rf_pred_2019

In [0]:
results_2019.head()

# Putting all the data together for visualization

In [0]:
results_2019['YEAR'] = 2019
results_2019.to_csv(root_path+'FrancoisWorkingFolder/predictions_for_2019.csv')